In [32]:
!pip install rdflib
!pip install SPARQLWrapper
!pip install owlready2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import FOAF, XMLNS, XSD, RDF, RDFS, OWL

g = Graph()
g.parse("assets_ontology_2.ttl")

print(len(g))


102


In [34]:
from rdflib import Namespace
import os
import json
import re


environments_count = 0


# Open asset file
f = open('/content/tk4tcfhfa.json')

data = json.load(f) # json file loaded as dictionary
n = Namespace("http://www.semanticweb.org/szm/megascan-assets-ontology#")

# For the code segment below, apply loop on all asset files in directory

# Create asset and set asset name
name = data["name"]     # obtain name of asset, convert it to RDF format literal
name_processed = re.sub("[^a-zA-Z0-9]", "_", name)
print("Created asset name = ", name)
print(name_processed)
asset = URIRef(str(n)+"Asset_"+name_processed)  # create a resource in the Assets Knowledge Graph (KG)
print("Asset = ", asset)
g.add((asset, RDF.type, n.Asset))
g.add((asset, n.assetName, Literal(name, datatype=XSD.string)))     # Add triple to KG: {new Asset resource, nameRelation (from schema), name obtained from asset file}

try:
  # Set asset ID
  id = Literal(str(data["id"]), datatype=XSD.string)     # obtain name of asset, convert it to RDF format literal
  g.add((asset, n.assetID, id))
  print("Added assetID = ", id)
except:
  print("No asset ID specified for asset", asset)

try:
  # Set asset search tags
  tags = data["tags"]
  print("number of tags = ", len(tags))
  for t in tags:
    tag = Literal(str(t), datatype=XSD.string)
    g.add((asset, n.assetTag, tag))
    print("Added tag = ", tag)

except:
  print("No search tags specified for asset", asset)

try:
  # Create category individuals and set their names, then assign asset categories
  categories = data["categories"]
  print("number of categories = ", len(categories))
  for c in categories:
    category = Literal(c, datatype=XSD.string)
    g.add((asset, n.assetCategory, category))
    print("Adding category = ", category)

except:
  print("No categories specified for asset", asset)


properties = data["properties"]
print("number of properties = ", len(properties))
for prop in properties:
  key = list(prop.keys())[0]
  value = list(prop.values())[0]
  print("prop['key'] = "  , prop["key"])
  print("prop['value'] = "  , prop["value"])

try:
  properties = data["properties"]
  print("number of properties = ", len(properties))
  for prop in properties:
    key = list(prop.keys())[0]
    value = list(prop.values())[0]

    print("prop['key'] = "  , prop["key"])
    print("prop['value'] = "  , prop["value"])

    print("prop key = ", key)
    if prop["key"] == "size":
      # print(value, Literal(value))
      if prop["value"] == "tiny":
        g.add((asset, n.assetSize, n.tiny))
        print("Added asset size = ", n.tiny)
      elif prop["value"] == "small":
        g.add((asset, n.assetSize, n.small))
        print("Added asset size = ", n.small)
      elif prop["value"] == "medium":
        g.add((asset, n.assetSize, n.medium))
        print("Added asset size = ", n.medium)
      elif prop["value"] == "large":
        g.add((asset, n.assetSize, n.large))
        print("Added asset size = ", n.large)
      else:
        g.add((asset, n.assetSize, n.extra_large))
        print("Added asset size = ", n.extra_large)

    elif prop["key"] == "age":
      age = Literal(prop["value"], datatype=XSD.string)
      g.add((asset, n.assetAge, age))
      print("Added asset age = ", age)

except:
  print("No properties specified for asset", asset)

try:
  # Set asset average color
  avg_color = Literal(str(data["averageColor"]), datatype=XSD.string)
  g.add((asset, n.assetAvgColor, avg_color))
  print("Added asset average color = ", avg_color)
except:
  print("No asset avg color specified for asset", asset)




unique_biomes = set()
unique_regions = set()

# Create biome individuals if not alrady created, then link to assets
try:
  biome = URIRef(str(n)+"Biome_"+data["environment"]["biome"])
  if biome not in unique_biomes:
    g.add((biome, RDF.type, n.Biome))
    biome_name = Literal(data["environment"]["biome"], datatype=XSD.string)
    g.add((biome, n.biomeName, biome_name))
    unique_biomes.add(biome)
    print("Added new biome to graph: ", biome)
  g.add((asset, n.assetEnvironmentBiome, biome))
  print(("Added asset's biome = ", biome))
except:
  print("No asset biome found for asset", asset)



# Create region individuals, then link to assets
try:
  region = URIRef(str(n)+"Region_"+data["environment"]["region"])
  if region not in unique_regions:
    g.add((region, RDF.type, n.Region))
    region_name = Literal(data["environment"]["region"], datatype=XSD.string)
    g.add((region, n.regionName, region_name))    
    unique_regions.add(region)
    print("Added new region to graph: ", region)
  g.add((asset, n.assetEnvironmentRegion, region))
  print(("Added asset's region = ", region))
except:
  print("No asset region found for asset", asset)


  # region = URIRef(str(n)+"Region_"+data["environment"]["region"])
  # if biome not in unique_biomes or region not in unique_regions:
  #   environments_count +=1
  #   environment = URIRef(str(n)+"Environment_"+str(environments_count))
  #   g.add((asset, n.assetHasEnvironment, environment))

  #   g.add((biome, RDFS.label, Literal(data["environment"]["biome"])))
  #   g.add((environment, n.environmentHasBiome, biome))

  #   g.add((region, RDFS.label, Literal(data["environment"]["region"])))
  #   g.add((environment, n.environmentHasRegion, region))

  # unique_biomes.add(biome)
  # unique_regions.add(region)
  
# except:
#   print("No environment with biome and region specified for asset", asset)


print(len(g))

f.close()

g.serialize(destination='Populated_Assets_KG.ttl', format='turtle')


Created asset name =  Mossy Mounds
Mossy_Mounds
Asset =  http://www.semanticweb.org/szm/megascan-assets-ontology#Asset_Mossy_Mounds
Added assetID =  tk4tcfhfa
number of tags =  14
Added tag =  ground
Added tag =  dirt
Added tag =  mound
Added tag =  embankment
Added tag =  terrain
Added tag =  moss
Added tag =  mossy
Added tag =  forest
Added tag =  lumps
Added tag =  rough
Added tag =  sweden
Added tag =  tyresta
Added tag =  archviz
Added tag =  nordic
number of categories =  3
Adding category =  3d
Adding category =  moss
Adding category =  ground
number of properties =  2
prop['key'] =  size
prop['value'] =  medium
prop['key'] =  mesh type
prop['value'] =  Open
number of properties =  2
prop['key'] =  size
prop['value'] =  medium
prop key =  key
Added asset size =  http://www.semanticweb.org/szm/megascan-assets-ontology#medium
prop['key'] =  mesh type
prop['value'] =  Open
prop key =  key
Added asset average color =  #38330E
Added new biome to graph:  http://www.semanticweb.org/szm

<Graph identifier=N09c814412a684a678a1a85a351514f3f (<class 'rdflib.graph.Graph'>)>

In [38]:
from rdflib import Namespace
import os
import json
import re


path_to_json_files = '/content/'
json_files = [pos_json for pos_json in os.listdir(path_to_json_files) if pos_json.endswith('.json')]
# print(json_files)  # for me this prints ['foo.json']

for asset_file in json_files:

  n = Namespace("http://www.semanticweb.org/szm/megascan-assets-ontology#")
  # Open asset file
  f = open(asset_file)
  data = json.load(f) # json file loaded as dictionary

  # Create asset and set asset name
  name = data["name"]     # obtain name of asset, convert it to RDF format literal
  name_processed = re.sub("[^a-zA-Z0-9]", "_", name)
  print("Created asset name = ", name)
  print(name_processed)
  asset = URIRef(str(n)+"Asset_"+name_processed)  # create a resource in the Assets Knowledge Graph (KG)
  print("Asset = ", asset)
  g.add((asset, RDF.type, n.Asset))
  g.add((asset, n.assetName, Literal(name, datatype=XSD.string)))     # Add triple to KG: {new Asset resource, nameRelation (from schema), name obtained from asset file}

  try:
    # Set asset ID
    id = Literal(str(data["id"]), datatype=XSD.string)     # obtain name of asset, convert it to RDF format literal
    g.add((asset, n.assetID, id))
    print("Added assetID = ", id)
  except:
    print("No asset ID specified for asset", asset)

  try:
    # Set asset search tags
    tags = data["tags"]
    print("number of tags = ", len(tags))
    for t in tags:
      tag = Literal(str(t), datatype=XSD.string)
      g.add((asset, n.assetTag, tag))
      print("Added tag = ", tag)

  except:
    print("No search tags specified for asset", asset)

  try:
    # Create category individuals and set their names, then assign asset categories
    categories = data["categories"]
    print("number of categories = ", len(categories))
    for c in categories:
      category = Literal(c, datatype=XSD.string)
      g.add((asset, n.assetCategory, category))
      print("Adding category = ", category)

  except:
    print("No categories specified for asset", asset)


  properties = data["properties"]
  print("number of properties = ", len(properties))
  for prop in properties:
    key = list(prop.keys())[0]
    value = list(prop.values())[0]
    print("prop['key'] = "  , prop["key"])
    print("prop['value'] = "  , prop["value"])

  try:
    properties = data["properties"]
    print("number of properties = ", len(properties))
    for prop in properties:
      key = list(prop.keys())[0]
      value = list(prop.values())[0]

      print("prop['key'] = "  , prop["key"])
      print("prop['value'] = "  , prop["value"])

      print("prop key = ", key)
      if prop["key"] == "size":
        # print(value, Literal(value))
        if prop["value"] == "tiny":
          g.add((asset, n.assetSize, n.tiny))
          print("Added asset size = ", n.tiny)
        elif prop["value"] == "small":
          g.add((asset, n.assetSize, n.small))
          print("Added asset size = ", n.small)
        elif prop["value"] == "medium":
          g.add((asset, n.assetSize, n.medium))
          print("Added asset size = ", n.medium)
        elif prop["value"] == "large":
          g.add((asset, n.assetSize, n.large))
          print("Added asset size = ", n.large)
        else:
          g.add((asset, n.assetSize, n.extra_large))
          print("Added asset size = ", n.extra_large)

      elif prop["key"] == "age":
        age = Literal(prop["value"], datatype=XSD.string)
        g.add((asset, n.assetAge, age))
        print("Added asset age = ", age)

  except:
    print("No properties specified for asset", asset)

  try:
    # Set asset average color
    avg_color = Literal(str(data["averageColor"]), datatype=XSD.string)
    g.add((asset, n.assetAvgColor, avg_color))
    print("Added asset average color = ", avg_color)
  except:
    print("No asset avg color specified for asset", asset)




  unique_biomes = set()
  unique_regions = set()

  # Create biome individuals if not alrady created, then link to assets
  try:
    biome = URIRef(str(n)+"Biome_"+data["environment"]["biome"])
    if biome not in unique_biomes:
      g.add((biome, RDF.type, n.Biome))
      biome_name = Literal(data["environment"]["biome"], datatype=XSD.string)
      g.add((biome, n.biomeName, biome_name))
      unique_biomes.add(biome)
      print("Added new biome to graph: ", biome)
    g.add((asset, n.assetEnvironmentBiome, biome))
    print(("Added asset's biome = ", biome))
  except:
    print("No asset biome found for asset", asset)



  # Create region individuals, then link to assets
  try:
    region = URIRef(str(n)+"Region_"+data["environment"]["region"])
    if region not in unique_regions:
      g.add((region, RDF.type, n.Region))
      region_name = Literal(data["environment"]["region"], datatype=XSD.string)
      g.add((region, n.regionName, region_name))    
      unique_regions.add(region)
      print("Added new region to graph: ", region)
    g.add((asset, n.assetEnvironmentRegion, region))
    print(("Added asset's region = ", region))
  except:
    print("No asset region found for asset", asset)

  print(len(g))

  f.close()

g.serialize(destination='Populated_Assets_KG.ttl', format='turtle')


Created asset name =  Wooden Chair
Wooden_Chair
Asset =  http://www.semanticweb.org/szm/megascan-assets-ontology#Asset_Wooden_Chair
Added assetID =  uknkaffaw
number of tags =  18
Added tag =  brown
Added tag =  rural
Added tag =  homes
Added tag =  dining-rooms
Added tag =  canteens
Added tag =  restaurants
Added tag =  houses
Added tag =  countryside
Added tag =  villages
Added tag =  wood
Added tag =  furniture
Added tag =  seat
Added tag =  rustic
Added tag =  wooden
Added tag =  armless
Added tag =  ladder-back
Added tag =  smooth
Added tag =  worn
number of categories =  4
Adding category =  3d
Adding category =  interior
Adding category =  furniture
Adding category =  seating
number of properties =  2
prop['key'] =  size
prop['value'] =  medium
prop['key'] =  mesh type
prop['value'] =  Closed
number of properties =  2
prop['key'] =  size
prop['value'] =  medium
prop key =  key
Added asset size =  http://www.semanticweb.org/szm/megascan-assets-ontology#medium
prop['key'] =  mesh 

<Graph identifier=N484a478076ee4f6182af495c15df4622 (<class 'rdflib.graph.Graph'>)>